In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier

from clulib import parse_data

In [2]:
energy = 2000
pidata, _, _ = parse_data('pi0', energy, 'Data5x5')
gadata, _, _ = parse_data('gamma', energy, 'Data5x5')
print(pidata.shape, gadata.shape)

D:\Ctau\Software\SCTParsimExamples\clulib.py:67: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(oneclu), np.array(twoclu), eventcnt


(74627, 2) (96212, 2)


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
def preprocess_data(data):
    X = data[:, :, 0]
    sums = X.sum(axis=1)
    return X / sums.reshape(-1, 1), sums
#     return X, sums

In [ ]:
def make_data(pidata, gadata):
    pidata0, epi = preprocess_data(pidata)
    gadata0, ega = preprocess_data(gadata)
    print(pidata0.shape)
    print(gadata0.shape)
    X = np.vstack((pidata0, gadata0))
    y = np.concatenate((
        np.zeros(pidata0.shape[0], dtype=int),
        np.ones(gadata0.shape[0], dtype=int)
    ))
    return train_test_split(X, y, test_size=0.2, random_state=47)

In [ ]:
X_train, X_test, y_train, y_test = make_data(pidata, gadata)
X_train.shape

In [ ]:
clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)

In [ ]:
plt.plot(clf.loss_curve_)

In [ ]:
clf.score(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
prob = clf.predict_proba(X_test)
prob[0]

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

In [ ]:
plt.hist(prob[y_test==1][:, 1], bins=100, histtype='step')
plt.hist(prob[y_test==0][:, 1], bins=100, histtype='step');

In [ ]:
prob_train = clf.predict_proba(X_train)

In [ ]:
plt.hist(prob_train[y_train==1][:, 1], bins=100, histtype='step')
plt.hist(prob_train[y_train==0][:, 1], bins=100, histtype='step');

In [ ]:
clf2 = MLPClassifier(
    random_state=1,
    hidden_layer_sizes=(50, 25),
    max_iter=300).fit(X_train, y_train)

In [ ]:
plt.plot(clf2.loss_curve_)

In [ ]:
roc_auc_score(y_test, clf2.predict_proba(X_test)[:, 1])

In [ ]:
prob2 = clf2.predict_proba(X_test)[:, 1]

In [ ]:
plt.hist(prob2[y_test==1], bins=100, histtype='step')
plt.hist(prob2[y_test==0], bins=100, histtype='step');

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gbc = GradientBoostingClassifier(random_state=9).fit(X_train, y_train)

In [ ]:
roc_auc_score(y_test, gbc.predict_proba(X_test)[:, 1])

In [ ]:
gbc_prob = gbc.predict_proba(X_test)[:, 1]

In [ ]:
plt.hist(gbc_prob[y_test==1], bins=100, histtype='step')
plt.hist(gbc_prob[y_test==0], bins=100, histtype='step');

In [ ]:
from sklearn.model_selection import learning_curve

In [ ]:
fitres = learning_curve(GradientBoostingClassifier(random_state=9), X_train, y_train)